# Samenhang

In [1]:
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats
import matplotlib.cm as cm
import statistics

missing_values = ['n/a', 'na', 'nan', 'N/A', 'NA', 'NaN', 'NAN', '--', 'Missing', 'missing', 'MISSING', 'Unknown',
                  'Weet ik niet', 'None', 'ONBEKEND', '"ontbrekend"', '"onbekend"', 'onbekend']

In [2]:
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import mean_squared_error, r2_score as r2score
class GeneralRegression:
    def __init__(self, degree=1, exp=False, log=False):
        self.degree = degree
        self.exp = exp
        self.log = log
        self.model = None
        self.x_orig = None
        self.y_orig = None
        self.X = None
        self.y = None

    def fit(self, x: np.array, y: np.array):
        self.x_orig = x
        self.y_orig = y
        self.X = x.reshape(-1, 1)

        if self.exp:
            self.y = np.log(y)

        else:
            self.y = y

        if self.log:
            self.X = np.log(self.X)

        self.model = make_pipeline(PolynomialFeatures(degree=self.degree), LinearRegression())
        self.model.fit(self.X, self.y)

    def predict(self, x: np.array):
        X = x.reshape(-1, 1)

        if self.exp:
            return np.exp(self.model.predict(X))

        if self.log:
            return self.model.predict(np.log(X))

        return self.model.predict(X)

    @property
    def r2_score(self):
        return r2score(self.y_orig, self.predict(self.x_orig))

    @property
    def se_(self):
        if self.exp:
            return mean_squared_error(self.predict(self.X), np.exp(self.y), squared=False)
        if self.log:
            return mean_squared_error(self.predict(self.X), np.log(self.y), squared=False)
        return mean_squared_error(self.predict(self.X), self.y, squared=False)

    @property
    def coef_(self):
        return self.model.steps[1][1].coef_

    @property
    def intercept_(self):
        return self.model.steps[1][1].intercept_

In [3]:
def outlier_boundaries(x, factor=1.5):
    Q1 = x.quantile(0.25)
    Q3 = x.quantile(0.75)
    I = Q3 - Q1
    return [Q1 - factor * I, Q3 + factor * I]


def drop_outliers(df, row):
    df['Z-Score'] = np.abs((df[row] - df[row].mean()) / df[row].std())
    filtered_df = df[df['Z-Score'] <= 3]
    filtered_df = filtered_df.drop('Z-Score', axis=1)
    return filtered_df


def aantal_klassen(series):
    n = len(series)
    print("Sturges (# klassen): " + str(math.ceil(1 + math.log2(n))))  # Sturges
    b = 3.5 * statistics.stdev(series) / (n ** (1 / 3))
    print("Scott (klasse breedte): " + str(math.ceil((series.max() - series.min()) / b)))  # Scott
    print("Scott (# klassen): " + str(math.ceil(math.sqrt(n))))
    print("[√n]: " + str(math.ceil(math.sqrt(n))))


def kendall(x, y):
    x = x.rank()
    y = y.rank()
    n = len(x)
    aantal = 0

    for i in range(1, n - 1):
        for j in range(i + 1, n):
            aantal = aantal + np.sign(x[i] - x[j]) * np.sign(y[i] - y[j])

    return 2 * aantal / (n / (n - 1))


def makescatter(x, y):
    plt.scatter(x, y)
    plt.xlabel(x.name)
    plt.ylabel(y.name)
    plt.show()

def scatterWhitReg(X,y,se=0): #model moet aanwezig zijn
    fig, ax = plt.subplots()
    _ = ax.scatter(X, y)
    xx = np.arange(X.min(), X.max(), (X.max() - X.min()) / 100)
    yy = model.predict(xx.reshape(-1, 1))
    _ = ax.fill_between(xx, yy - se, yy + se, color="yellow", alpha=0.2)
    _ = ax.plot(xx, yy, color='red')
    _ = ax.set_xlabel(X.name)
    _ = ax.set_ylabel(y.name)
    plt.show()

## Stress en weersomstandigheden
In deze studie vragen we ons af welke invloed het weer heeft op het stressgevoel. We vroegen een aantal
personen om iedere dag hun stressgevoel te noteren. Bij deze gegevens noteerden we de gemiddelde
temperatuur, luchtvochtigheid en hoeveelheid neerslag op die dag.

1. lees de data in. Welke kolommen bevatten NA-waarden? Verwijder de rijen met NA-waarden.
Verwijder de rijen met extreme uitschieters bij neerslag. De luchtvochtigheid mag niet hoger zijn dan 100. Verwijder de rijen waarbij dat zo is. Hoeveel rijen hou je nu over?

In [18]:
df = pd.read_csv("./data/stress.csv", sep="\t", decimal=",", header=0, na_values=missing_values)
df.drop(df[df['vochtigheid']>100].index,inplace=True)
df.dropna(inplace=True)

# # Filter de rijen met uitschieters
# outliers_neerslag = outlier_boundaries(df.neerslag)
# df = df[(df.neerslag >= outliers_neerslag[0]) & (df.neerslag <= outliers_neerslag[1])]

df = drop_outliers(df, "neerslag")
print(len(df))

1090


2. Welk meetniveau hebben de kolommen?

Stress: ordinaal
Temp: interval
Neerslag: ratio
Vochtigheid: ratio

3. Wat is de gemiddelde temperatuur, luchtvochtigheid en neerslag?

In [20]:
gem_temp = df.temp.mean().round(2)
gem_vocht = df.vochtigheid.mean().round(2)
gem_neerslag = df.neerslag.mean().round(2)
print(gem_temp, gem_vocht, gem_neerslag)

10.32 79.88 2.37


4. Zoek de correlatie tussen stress en temperatuur. Welke methode gebruik je best? Welke waarde
vind je? Wat betekent deze?

In [25]:
print(df.stress.corr(df.temp, method="kendall"))

print("Aangezien stress ordinaal is en temperatuur interval, kunnen we Pearson niet gebruiken.\nEen betere maat is Kendall. een sterke positieve lineaire correlatie")

0.6533314631732314
Aangezien stress ordinaal is en temperatuur interval, kunnen we Pearson niet gebruiken.
Een betere maat is Kendall. een sterke positieve lineaire correlatie


5. Zoek de correlatie tussen stress en luchtvochtigheid. Welke waarde vind je? Wat betekent deze?

In [27]:
print(df.stress.corr(df.vochtigheid, method="kendall"))
print("Dit betekent een negatief zwak lineair verband.")

-0.3220206998144224
Dit betekent een negatief zwak lineair verband.


6. Zoek de correlatie tussen stress en neerslag. Welke waarde vind je? Wat betekent deze?

In [29]:
print(df.stress.corr(df.neerslag, method="kendall"))
print("Dit betekent geen lineair verband.")

0.0032704318362192096
Dit betekent geen lineair verband.


7. Welke factoren spelen dus een rol in het stressgevoel?

Temperatuur speelt dus duidelijk een rol op het stressgevoel. Luchtvochtigheid
slechts in heel kleine mate